# Second Project: Parser

The second project requires you to implement a Parser for the uC language and
generate a parse tree as output (note that the Abstract Syntax Tree will be
built only in the third project). To complete this second project, you will also
use the [PLY](http://www.dabeaz.com/ply/), a Python version of the
[lex/yacc](http://dinosaur.compilertools.net/) toolset with the same
functionality but with a friendlier interface. Please read the complete
contents of this section and carefully complete the steps indicated.

# Parser at a glance: Building a Parser Tree
Consider the following grammar. We will develop a parser to recognize sentences in this
grammar and build a parse tree for them. In this grammar, the following syntax is used:
```
 { symbols }*  --> Zero or more repetitions of symbols
 { symbols }+  --> One or more repetitions of symbols
 { symbols }?  --> Zero or one occurences of symbols (optional)
 sym1 | sym2   --> Either sym1 or sym2 (a choice)
```

In [ ]:
# Grammar
'''
program : statements EOF

statements : { statement }+

statement : assign_statement
          | if_statement
          | while_statement
          | print_statement

assign_statement : ID EQUALS expr SEMI

if_statement : IF expr LBRACE statements RBRACE { ELSE LBRACE statements RBRACE }?

while_statement : WHILE expr LBRACE statements RBRACE

print_statement : PRINT expr SEMI

expr : expr PLUS expr
     | expr TIMES expr
     | expr EQ expr
     | expr NE expr
     | NUM
     | ID
     | LPAREN expr RPAREN
'''

 Example of valid sentences for this grammar

## Lexer
The first step is to build a lexical analyzer for the terminals of this grammar:

In [ ]:
from ply.lex import lex

# tokens
tokens = ('ID', 'NUM', 'PLUS', 'TIMES', 'NE', 'EQ', 'EQUALS', 'SEMI', 'LPAREN',
          'RPAREN', 'LBRACE', 'RBRACE', 'PRINT', 'IF', 'ELSE', 'WHILE')


def t_ID(t):
    r'[a-zA-Z_][a-zA-Z0-9_]*'
    if t.value == 'print':
        t.type = "PRINT"
    elif t.value == 'if':
        t.type = "IF"
    elif t.value == 'else':
        t.type = "ELSE"
    elif t.value == 'while':
        t.type = "WHILE"
    return t


def t_NUM(t):
    r'[0-9]+'
    t.value = int(t.value)
    return t


t_PLUS = r'\+'
t_TIMES = r'\*'
t_NE = r'!='
t_EQ = r'=='
t_EQUALS = r'='
t_SEMI = r';'
t_LPAREN = r'\('
t_RPAREN = r'\)'
t_LBRACE = r'{'
t_RBRACE = r'}'


def t_error(t):
    print(f'Illegal character {t.value[0]}')
    t.lexer.skip(1)


t_ignore = ' \t'
__file__ = 'parser_at_a_glance.ipynb'  # necessário apenas no contexto do jupyter
lexer = lex()
lexer.input("a = 3 * 4 + 5")
for tok in lexer:
    print(tok)

## Recognizing the sentences

In [ ]:
from ply.yacc import yacc


def p_program(p):
    ''' program : statements
    '''


def p_statement_list(p):
    ''' statements : statements statement
                   | statement
    '''


def p_assign_statement(p):
    ''' statement : ID EQUALS expr SEMI
    '''


def p_if_statement_1(p):
    ''' statement : IF expr LBRACE statements RBRACE
    '''


def p_if_statements_2(p):
    ''' statement : IF expr LBRACE statements RBRACE ELSE LBRACE statements RBRACE
    '''


def p_while_statement(p):
    ''' statement : WHILE expr LBRACE statements RBRACE
    '''


def p_print_statement(p):
    ''' statement : PRINT expr SEMI
    '''


def p_binop_expr(p):
    ''' expr : expr PLUS expr
             | expr TIMES expr
             | expr EQ expr
             | expr NE expr
    '''


def p_num_expr(p):
    ''' expr : NUM
    '''


def p_name_expr(p):
    ''' expr : ID
    '''


def p_compound_expr(p):
    ''' expr : LPAREN expr RPAREN
    '''


parser = yacc(write_tables=False)

Let's ignore the warnings for now and try to parser for a valid sentence:

In [ ]:
parser.parse('a = 3 * 4 + 5 ;')

 Nothing happened! Let's see what happens to an invalid sentence:

In [ ]:
parser.parse('a == 3 ;')

Now let's add information to build a parse tree:

In [ ]:
from ply.yacc import yacc


def p_program(p):
    ''' program : statements
    '''
    p[0] = ('program', p[1])


def p_statement_list(p):
    ''' statements : statements statement
                   | statement
    '''
    p[0] = p[1] if len(p) == 2 else p[1] + (p[2])


def p_assign_statement(p):
    ''' statement : ID EQUALS expr SEMI
    '''
    p[0] = ('assign', p[1], p[3])


def p_if_statement_1(p):
    ''' statement : IF expr LBRACE statements RBRACE
    '''
    p[0] = ('if', p[2], p[4], None)


def p_if_statements_2(p):
    ''' statement : IF expr LBRACE statements RBRACE ELSE LBRACE statements RBRACE
    '''
    p[0] = ('if', p[2], p[4], p[8])


def p_while_statement(p):
    ''' statement : WHILE expr LBRACE statements RBRACE
    '''
    p[0] = ('while', p[2], p[4])


def p_print_statement(p):
    ''' statement : PRINT expr SEMI
    '''
    p[0] = ('print', p[2])


def p_binop_expr(p):
    ''' expr : expr PLUS expr
             | expr TIMES expr
             | expr EQ expr
             | expr NE expr
    '''
    p[0] = (p[2], p[1], p[3])


def p_num_expr(p):
    ''' expr : NUM
    '''
    p[0] = ('num', p[1])


def p_name_expr(p):
    ''' expr : ID
    '''
    p[0] = ('id', p[1])


def p_compound_expr(p):
    ''' expr : LPAREN expr RPAREN
    '''
    p[0] = p[2]


parser = yacc(write_tables=False)

In [ ]:
parser.parse("a = 3 * 4 + 5 ;")

Built the parser tree we noticed that it is not respecting the precedence of the
operators. We need to indicate this in the program. Let's also take advantage
and define an error routine for the parser:

In [ ]:
from ply.yacc import yacc


def p_program(p):
    ''' program : statements
    '''
    p[0] = ('program', p[1])


def p_statement_list(p):
    ''' statements : statements statement
                   | statement
    '''
    p[0] = p[1] if len(p) == 2 else p[1] + (p[2])


def p_assign_statement(p):
    ''' statement : ID EQUALS expr SEMI
    '''
    p[0] = ('assign', p[1], p[3])


def p_if_statement_1(p):
    ''' statement : IF expr LBRACE statements RBRACE
    '''
    p[0] = ('if', p[2], p[4], None)


def p_if_statements_2(p):
    ''' statement : IF expr LBRACE statements RBRACE ELSE LBRACE statements RBRACE
    '''
    p[0] = ('if', p[2], p[4], p[8])


def p_while_statement(p):
    ''' statement : WHILE expr LBRACE statements RBRACE
    '''
    p[0] = ('while', p[2], p[4])


def p_print_statement(p):
    ''' statement : PRINT expr SEMI
    '''
    p[0] = ('print', p[2])


def p_binop_expr(p):
    ''' expr : expr PLUS expr
             | expr TIMES expr
             | expr EQ expr
             | expr NE expr
    '''
    p[0] = (p[2], p[1], p[3])


def p_num_expr(p):
    ''' expr : NUM
    '''
    p[0] = ('num', p[1])


def p_name_expr(p):
    ''' expr : ID
    '''
    p[0] = ('id', p[1])


def p_compound_expr(p):
    ''' expr : LPAREN expr RPAREN
    '''
    p[0] = p[2]


def p_error(p):
    if p:
        print("Error near the symbol %s" % p.value)
    else:
        print("Error at the end of input")


precedence = (
    ('left', 'PLUS'),
    ('left', 'TIMES'),
    ('left', 'EQ'),
    ('left', 'NE')
    )

parser = yacc(write_tables=False)

With the definition of operator precedence, shift-reduce conflicts were resolved.
Let's run the parser for our example sentences again:

In [ ]:
parser.parse('a = 3 * 4 + 5 ;')

In [ ]:
parser.parse('a = 3 * ;')

In [ ]:
parser.parse('a == 3 ; ')

In [ ]:
parser.parse('print a ;')

In [ ]:
code = '''
a = 3;
b = 4 * a;
print (a+b);
'''

In [ ]:
parser.parse(code.__str__().replace('\n', ' '))

In [ ]:
code = '''
count = 0;
max = 5;
while count != max {
  count = count + 1;
  print count;
}
'''

In [ ]:
parser.parse(code.__str__().replace('\n', ' '))

## Writing a Parser for the uC language

In this step, you must write the basic shell of a parser for the uC language.
A formal BNF of the language is [here](./doc/uC_Grammar.ipynb). Your task is to write parsing
rules using PLY
(see [PLY-Yacc](http://www.dabeaz.com/ply/ply.html#ply_nn22) documentation).

Your task is translate the BNF into a collection of parser functions. For example, a rule
such as :
```
  <program> ::= {<global_declaration>}+
```  
Gets turned into a Python function of the form:

In [ ]:
class Parser:
    ...
    def p_program(self, p):
        """ program  : global_declaration_list
        """
        p[0] = ('Program', p[1])

    def p_global_declaration_list(self, p):
        """ global_declaration_list : global_declaration
                                    | global_declaration_list global_declaration
        """
        p[0] = [p[1]] if len(p) == 2 else p[1] + [p[2]]

In the body of each rule, create an appropriate name and assign it to p[0] as shown
above.

For the purposes of lineno number tracking, you should assign a line number to each
production as appropriate. See http://www.dabeaz.com/ply/ply.html#ply_nn33. To do this,
I suggest pulling the line number off of any nearby terminal symbol. For example:

In [ ]:
    def p_identifier(self, p):
        """ identifier : ID """
        p[0] = ('ID', p[1])
        p.set_lineno(0, p.lineno(1))

Your goal, at the end of this second project, is to **syntactically** recognize programs
expressed in the uC language. You can use as a basis the examples contained
[here](./doc/uC_Grammar.ipynb). For this, the ideal is that you get your grammar to present
**only one** shift-reduce conflict that is relative to the if-else sentence.

Suggestion: You should start simple and incrementally work your way up to building the
complete grammar.